# Van Price Prediction

One of the requirements is that all or at least part of the data has to be obtained through webscraping or APIs.

Once I have decided the topic of my project, I look at possible websites to see how they structure the information and then decide which one to webscrape. 

## Possible websites

https://www.milanuncios.com/furgonetas-de-segunda-mano-en-barcelona/?demanda=n&orden=relevance&fromSearch=1&hitOrigin=listing

https://www.mobile.de/es/categor%C3%ADa/veh%C3%ADculo/vhc:car,dmg:false,vcg:van

https://www.autohero.com/es/search/?bodyType=van_transporter

**Features:** brand, model, mileage, year (age), fuel, doors, horsepower (cv)...

# Web analysis

## Mobile.de

https://www.mobile.de/es/b%C3%BAsqueda-detallada/vhc:car,cnd:used,stx:2,dmg:false,vcg:van/pg:dspcar

I decide to webscrape this german website, because it has a lot of filters and the information is really structured (like everything in Germany).

This is what I thought I could get after analyzing the website.

**Columns**: 
1. url?
2. title: ``brand``, ``model``, size?
3. ``year`` 
4. ``kms``
5. ``price``
6. ``power``: (cv/HP)
7. ``fuel``: only about 2% are electric, so I'm gonna put a filter to keep only the ones that run with diesel or gasoline.

Dins

8. ``Primer_registro``: month and year of the first registration
9. ``engine_displacement``: related to the horsepower, maybe I won't use it, but I prefer to scrape it just in case. Put > 1000cm3 as a filter
10. ``seats``: I have to add a filter as well because it's not always shown
- 2: small cargo van
- 3: medium/big cargo van
- 4: medium/big passenger van
- 5: small passenger van
11. ``owners``: I have to add a filter of 4 owners, because it's not always shown. This filter doesn't mean that only cars with 4 owners will appear
12. ``Energieverbrauch (comb.)1``: consumption (L/100km). I have to add a filter as well because it's not always shown (min 3L/100km)
13. ``sliding_doors``: not always shown. If I have any option I can filter out cars.

**Filters**:

- Second hand vehicles to exclude new vehicles.
- Type: van
- ``fuel``: only Diesel and Gasoline, which make almost 98% of the vehicles of the website
- ``consumption``: minimum of 3L/100km.
- ``seats``: minimum 2 seats.
- ``owners``: 4 
- ``sliding doors``: mark any option (right, left and both-sided) to exclude cars
- Adds with images (in case I need to check information later, like the model)
- Damaged vehicles: don't show

https://www.mobile.de/es/categor%C3%ADa/veh%C3%ADculo/vhc:car,srt:date,sro:desc,ful:diesel!petrol,cncn:3,ccn:1000,sld:sliding_door_left,cnd:used,stn:2,dmg:false,vcg:van,ao:pictures,pvo:4

All these filters applied (sliding door = left)

# Webscraping

In [3]:
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
import time
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sys

## Code changing pages

In [ ]:
# dfconcat = pd.read_csv('Datasets/dfconcat.csv') # in case I need to import the last df I scrapped

In [ ]:
PATH = ("C:/Program Files (x86)/chromedriver.exe")
driver = webdriver.Chrome(PATH) # define the webdriver

# create empty lists
titles = []
prices = []
dates = []
fuels = []
kms = []
powers = []
capacities = []
seats = []
owners = []
consumptions = []
sliding_doors = []

# creating an empty df
df = pd.DataFrame({'title': titles, 'price': prices, 'date': dates, 'fuel': fuels, 'km': kms, 'power_cv': powers, 'displacement_cm3': capacities, 
        'consumption': consumptions, 'seats': seats, 'owners': owners, 'sliding_doors': sliding_doors})

# tracking the number of vans scrapped (or at least attempted)
count = 0

for page in range (1,40): # there are many pages, in case it doesn't find the next button, so that it changes to the next page
    print(f'Page: {page}')
    # the main url I have to update everytime I run the code
    url = f'https://www.mobile.de/es/categor%C3%ADa/veh%C3%ADculo/vhc:car,srt:mileage,sro:desc,ful:diesel!petrol,cncn:3,mln:8000,mlx:9999,ccn:1000,sld:sliding_door_right,cnd:used,stn:2,dmg:false,vcg:van,ao:pictures,pvo:4'
    driver.get(url)

    time.sleep(1)
    try: # accept the cookies
        aceptar = driver.find_element_by_class_name('sc-braxZu')
        aceptar.click()
    except:
        pass

    first_result = driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/section/div[1]/article[1]/div/div[2]/a')
    first_result.click() # click on the first advertisment on the first page to start scraping
    time.sleep(1)

    while True: # the title is in 2 formats
        try:
            # Attempt to locate the first format
            title = driver.find_element_by_css_selector(
                'body > div.g-content > div > section.vehicle-details.g-row.vehicle-details-premium > div.vehicle-details-main.g-col-m-8 > div.vip-box.cBox.cBox--content.u-clearfix > div.g-row.u-margin-bottom-9'
            ).text
        except:
            # if cannot find the first format, try the second format
            title = driver.find_element_by_css_selector(
                'body > div.g-content > div > section.vehicle-details.g-row > div.vehicle-details-main.g-col-m-8 > div.vip-box.cBox.cBox--content.u-clearfix > div.g-row.u-margin-bottom-9 > h1'
            ).text

        # defining the price and the date 
        price = driver.find_element_by_css_selector('body > div.g-content > div > section.vehicle-details.g-row > div.vehicle-details-main.g-col-m-8 > div:nth-child(3) > div > div > div:nth-child(1) > div > span')
        price_val = int(price.text.split(' ')[0].replace('.', ''))
        date = (driver.find_element_by_css_selector('body > div.g-content > div > section.vehicle-details.g-row > div.vehicle-details-main.g-col-m-8 > div:nth-child(5) > div > div:nth-child(1) > div > div.attributes-box.g-col-12 > div:nth-child(3) > span.g-col-6.u-text-bold')).text

        # Check if the combination of title, price, and date exist in the same row in the DataFrame I'm currently scraping (df) or the one containing all the vans I have already scraped (dfconcat)
        van_in_df = ((df['title'] == title) & (df['price'] == price_val) & (df['date'] == date)).any()
        van_in_dfconcat = ((dfconcat['title'] == title) & (dfconcat['price'] == price_val) & (dfconcat['date'] == date)).any() # this only works if dfconcat is already imported
        if van_in_df:
            print('Van already in df') # and don't scrape so I don't waste time
        elif van_in_dfconcat:
            print('Van already in dfconcat') # and don't scrape so I don't waste time
        else: # scrape the info of the van
            titles.append(title)
            prices.append(price_val)
            dates.append(date)
            
            classes = driver.find_elements_by_class_name('g-col-6')
            for i in range(len(classes)):
                if classes[i].text == 'Combustible':
                    fuels.append(classes[i+1].text.split(',')[0])
                elif classes[i].text == 'Kilometraje':
                    km = int(classes[i+1].text.split(' ')[0].replace('.',''))
                    kms.append(km)
                elif classes[i].text == 'Capacidad cúbica':
                    capacities.append(int(classes[i+1].text.split(' ')[0].replace('.','')))
                elif classes[i].text == 'Sliding door':
                    sliding_doors.append(classes[i+1].text.split(' ')[2])
            
            # handling the 2 values sometimes are missing
            found_consumption = False
            found_power = False

            for i in range(len(classes)):
                if classes[i].text == 'Energieverbrauch (comb.)1':
                    consumptions.append(classes[i+1].text.split(' ')[0].replace(',','.'))
                    found_consumption = True

                # Check for consumption
                elif classes[i].text == 'Potencia':
                    powers.append(classes[i+1].text.split(' ')[0])
                    found_power = True

            # handling power not found
            if not found_power:
                print("Power not found, appending NaN.") # print it so I can see it
                powers.append(np.nan)

            # handling consumption not found
            if not found_consumption:
                print("Consumption not found, appending NaN.") # print it so I can see it
                consumptions.append(np.nan)
        
            seat = driver.find_element_by_css_selector('body > div.g-content > div > section.vehicle-details.g-row > div.vehicle-details-main.g-col-m-8 > div:nth-child(5) > div > div:nth-child(1) > div > div.further-tec-data.g-col-12 > div:nth-child(2) > span.g-col-6.u-text-bold')
            seats.append(seat.text)
            owner = driver.find_element_by_css_selector('body > div.g-content > div > section.vehicle-details.g-row > div.vehicle-details-main.g-col-m-8 > div:nth-child(5) > div > div:nth-child(1) > div > div.further-tec-data.g-col-12 > div:nth-child(4) > span.g-col-6.u-text-bold')
            owners.append(owner.text)

            lists = [titles, prices, dates, fuels, kms, powers, capacities, seats, owners, consumptions, sliding_doors]
            list_lengths = [len(lst) for lst in lists]
            all_same_length = all(length == list_lengths[0] for length in list_lengths)

            if all_same_length:
                print("Alles gut")
            else:
                print("Lists have different lengths:", list_lengths) # theorically this should not happen, but in case it does, I want it to stop scraping because that would be mean I have some info missing
                sys.exit()
        
        # creating the new dataframe with the scraped vans
        df = pd.DataFrame({'title': titles, 'price': prices, 'date': dates, 'fuel': fuels, 'km': kms, 'power_cv': powers, 'displacement_cm3': capacities, 
                'consumption': consumptions, 'seats': seats, 'owners': owners, 'sliding_doors': sliding_doors})
        count +=1
        print(f'{count} attempts, {len(titles)} scrapped, min km: {df.km.min()}') # to compare the number of attempts with the number of succesfully scraped vans

        # I scrape in descending order of kms, so if the km of the current van are higher than the min(km) of the df, it means it will try to scrape vans I have already scraped
        if km > df.km.min():
            print("It looks like it's trying to scrape a van already scraped") 
            sys.exit()
        else:
            try:
                next = driver.find_element_by_class_name('gicon-next-s')
                next.click() # click on the next button to scrape the next van
                time.sleep(1)
            except:
                break # if there is no next button found, stop scraping

Page: 1


c:\Users\Arnaldur\anaconda3\Lib\site-packages\selenium\webdriver\remote\webdriver.py:346: SyntaxWarning: invalid escape sequence '\_'
  """Finds an element by id.
c:\Users\Arnaldur\anaconda3\Lib\site-packages\selenium\webdriver\remote\webdriver.py:363: SyntaxWarning: invalid escape sequence '\_'
  """
c:\Users\Arnaldur\anaconda3\Lib\site-packages\selenium\webdriver\remote\webdriver.py:617: SyntaxWarning: invalid escape sequence '\*'
  """
c:\Users\Arnaldur\anaconda3\Lib\site-packages\selenium\webdriver\remote\webdriver.py:639: SyntaxWarning: invalid escape sequence '\*'
  """


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.265)


## Creating df, exporting and importing csv, pd.concat

In [154]:
lists = (titles, prices, dates, fuels, kms, powers, capacities, seats, owners, consumptions, sliding_doors)
for i in lists:
    print(len(i)) # check that all lists have the same length (even though I had make sure with the previous code)

315
315
315
315
315
315
315
315
315
315
315


In [68]:
# if I stop the scraping in the middle of a van and from the last van I have info only of some variables, I want to delete them, so I keep only the vans with the info of all the variables
lists = (titles, prices, dates, fuels, kms, capacities, seats, owners, consumptions, sliding_doors)
# for i in lists:
#     del i[-1]

In [13]:
df # the df I just scrapped

,title,price,date,fuel,km,power_cv,displacement_cm3,consumption,seats,owners,sliding_doors
0,Volkswagen T7 Multivan Basis KÜ Navi LED App C...,50898,05/2024,Diésel,9999,110,1968,6.5,7,1,both-sided
1,Mercedes-Benz V 300 d Avantgarde MOPF AHK/AMG/...,96970,11/2024,Diésel,9999,174,1950,8.2,7,1,both-sided
2,Mercedes-Benz V 300 Avantgarde AMG lang STANDH...,93736,08/2024,Diésel,9999,174,1950,7.5,7,1,both-sided
3,Mercedes-Benz V 220 d Marco Polo MOPF ALLRAD/M...,79390,11/2024,Diésel,9999,120,1950,8.2,4,1,right


### dfconcat

In [157]:
df_vans = pd.read_csv('Datasets/dfconcat.csv') # import the df that I scrapped previously so I can concatenate it with the one I just scrapped
df_vans.drop(columns=['Unnamed: 0'], inplace=True)
df_vans

,title,price,date,fuel,km,power_cv,displacement_cm3,consumption,seats,owners,sliding_doors
0,Mercedes-Benz Vito Kasten 122 CDI lang Austaus...,8999,07/2013,Diésel,551000,165.0,2987,8.6,2,2,both-sided
1,Mercedes-Benz V 220 *Extra Lang*Kamera *PDC *Navi,20800,11/2016,Diésel,539000,120.0,2143,6.1,8,2,both-sided
2,Mercedes-Benz V 250 d Avantgarde Edition lang ...,19900,03/2016,Diésel,531766,140.0,2143,6.0,6,1,both-sided
3,Mercedes-Benz V -Klasse V 250 CDI/BT/d AVANTGARD,28600,06/2018,Diésel,520093,140.0,2143,6.0,8,1,both-sided
4,"Mercedes-Benz V 200 V/220 CDI/d, 250 CDI/BT/d ...",29990,12/2017,Diésel,519211,140.0,2143,6.0,8,1,both-sided
...,...,...,...,...,...,...,...,...,...,...,...
21418,Volkswagen Multivan 2.0 TDI SCR DSG Life Navi ...,61990,06/2024,Diésel,10000,110.0,1968,6.6,7,1,both-sided
21419,Ford Tourneo Custom Bus 320 L2 Tourneo neues M...,54950,03/2024,Diésel,10000,110.0,1996,8.7,8,1,both-sided
21420,Volkswagen CADDY 2.0 TDI Rollstuhlumbau,38290,06/2024,Diésel,10000,75.0,1968,5.3,5,1,both-sided
21421,Toyota CROSSCAMP Flex Xenon Navi AHK Standheizung,58990,04/2024,Diésel,10000,106.0,1997,5.6,5,1,both-sided


In [ ]:
dfconcat = pd.concat([df_vans, df]) # the concat of the df I just scrapped and the vans I had already scrapped before
dfconcat.reset_index(drop=True, inplace=True)
dfconcat

,title,price,date,fuel,km,power_cv,displacement_cm3,consumption,seats,owners,sliding_doors
0,Mercedes-Benz Vito Kasten 122 CDI lang Austaus...,8999,07/2013,Diésel,551000,165.0,2987,8.6,2,2,both-sided
1,Mercedes-Benz V 220 *Extra Lang*Kamera *PDC *Navi,20800,11/2016,Diésel,539000,120.0,2143,6.1,8,2,both-sided
2,Mercedes-Benz V 250 d Avantgarde Edition lang ...,19900,03/2016,Diésel,531766,140.0,2143,6.0,6,1,both-sided
3,Mercedes-Benz V -Klasse V 250 CDI/BT/d AVANTGARD,28600,06/2018,Diésel,520093,140.0,2143,6.0,8,1,both-sided
4,"Mercedes-Benz V 200 V/220 CDI/d, 250 CDI/BT/d ...",29990,12/2017,Diésel,519211,140.0,2143,6.0,8,1,both-sided
...,...,...,...,...,...,...,...,...,...,...,...
21733,Opel Vivaro 2.0 D Cargo M,31500,03/2024,Diésel,10000,106,2000,NaN,3,1,right
21734,"Renault Trafic Combi L2H1 3,0t Life 2.0 BLUE d...",36890,07/2024,Diésel,10000,81,1995,7.1,9,1,right
21735,Renault Kangoo Rapid Blue dCi 75 Advance,21950,01/2024,Diésel,10000,55,1461,5.2,2,1,right
21736,"Volkswagen T6 California T6.1 Ocean Navi,Sport...",73900,12/2023,Diésel,10000,110,1968,NaN,4,1,right


In [167]:
dfconcat.to_csv('Datasets/dfconcat.csv') # export to csv

In [5]:
dfconcat = pd.read_csv('Datasets/dfconcat.csv') # in case I need to import the last df I scrapped

In [160]:
df.km.min() # I sort the values by km and descending order, so I know where I have to continue scraping

10000

In [12]:
dfconcat.km.min() # I sort the values by km and descending order, so I know where I have to continue scraping

10000

In [166]:
dfconcat.sliding_doors.value_counts() 

sliding_doors
right         11012
both-sided    10360
left            366
Name: count, dtype: int64